In [2]:
from rl.dynamic_programming import V, S, A
from rl import dynamic_programming

In [3]:
from rl import markov_process, markov_decision_process

In [4]:
import operator
from typing import Mapping, Iterator, TypeVar, Tuple, Dict, Iterable

import numpy as np

from rl.distribution import Categorical, Choose
from rl.iterate import converged, iterate
from rl.markov_process import NonTerminal, State
from rl.markov_decision_process import (FiniteMarkovDecisionProcess,
                                        FiniteMarkovRewardProcess)
from rl.policy import FinitePolicy, FiniteDeterministicPolicy

In [5]:
from rl.midterm_2022.priority_q import  PriorityQueue
from rl.midterm_2022 import grid_maze

In [6]:
import importlib

In [7]:
from time import time

# Problem 1

In this problem we will implement the gaps-based value iteration algorithm mentioned in class.

The gaps-based iteration algorithm proceeds as follows

1. Initialize the value function to zero for all states: $v[s] = 0\ \forall s \in \mathcal{N}$
2. Calculate the gaps for each state: $g[s] = |v[s] - \max_a \mathcal{R}(s,a) + \sum_{s'} \mathcal{P}(s,a,s') \cdot v(s')|$
3. While there is some gap that exceeds a threshold
 - Select the state with the  largest gap: $s_{max} = \arg\max_{s \in \mathcal{N}} g[s]$
 - Update the value function for $s_{max}$: $v[s_{max}] = \max_a \mathcal{R}(s_{max},a) + \sum_{s'}\mathcal{P}(s_{max},a,s') \cdot v(s')$
 -  Update the gap for $s_{max}$: $g[s_{max}] = 0$
4. Return v

We will test your implementation on a grid maze MDP. We have defined this class in "grid_maze.py", you should  briefly familiarize yourself with that code. In particular pay attention to the difference in reward functions for the two classes "GridMazeMDP_Dense" and "GridMazeMDP_Sparse"

Here is how you can use the classes:

In [8]:
underlying_maze = grid_maze.Maze(10, 10)
maze_mdp = grid_maze.GridMazeMDP_Sparse(underlying_maze, 0, 0)

you can visualize the maze if you wish

In [9]:
print(maze_mdp)

--------------------
|*  |     |   | |   |
| + +-+ +-+-+ + +-+ +
| | | | | |         |
| +-+ + + + +-+-+-+-+
|           | | | | |
| + + + + + + + + + +
| | | | | |         |
| + +-+ +-+ +-+ + + +
| |   |   | |   | | |
| + + +-+ +-+-+-+ +-+
| | |   |   |       |
|-+-+-+-+ +-+-+-+ +-+
|           | |   | |
|-+-+ + +-+ + +-+ + +
|     | | | |       |
|-+-+ + + + +-+-+ + +
|   | |   |   | | | |
| + +-+ +-+ +-+ +-+-+
| |     |           |
|-+-+-+-+-+-+-+-+-+-+


you can also visualize a policy on the mdp

In [10]:
v2_res = dynamic_programming.value_iteration_result(maze_mdp, 0.9)
print(maze_mdp.print_policy(v2_res[1]))

--------------------
|* <|> v <|> v|v|> v|
| + +-+ +-+-+ + +-+ +
|^|^|v|v|v|v < < < <|
| +-+ + + + +-+-+-+-+
|^ < < < < <|v|v|v|v|
| + + + + + + + + + +
|^|^|^|^|^|^ < < < <|
| + +-+ +-+ +-+ + + +
|^|^ <|^ <|^|> ^|^|^|
| + + +-+ +-+-+-+ +-+
|^|^|^ <|^ <|> > ^ <|
|-+-+-+-+ +-+-+-+ +-+
|> > > > ^ <|v|> ^|v|
|-+-+ + +-+ + +-+ + +
|> > ^|^|v|^|> > ^ <|
|-+-+ + + + +-+-+ + +
|> v|^|^ <|^ <|v|^|^|
| + +-+ +-+ +-+ +-+-+
|^|> > ^|> ^ < < < <|
|-+-+-+-+-+-+-+-+-+-+


You will need to make use of the PriorityQueue class in your implementation. A PriorityQueue is an ordered queue which supports the following operations
1. isEmpty(self): check if the queue is empty   
2. contains(self, element): check if the queue contains an element
3. peek(self): peek at the highest priority element in the queue    
4. pop(self): remove and return the highest priority element in the queue    
5. insert(self, element, priority): insert an element into the queue with given priority
6. update(self, element, new_priority): update the priority of an element in the queue
7. delete(self, element): delete an element from the queue

Below are some examples of using the queue

In [11]:
q: PriorityQueue = PriorityQueue()
print(q.isEmpty(), ':', "the queue is empty")
q.insert("a", 1)
print(q.isEmpty(), ':',  "the queue is not empty")
print(q.contains("a"), ':',  "the queue contains a")
print(q.contains("b"), ':',  "the queue does not contain a")
print(q.peek(), ':',  "a is the first element in the queue")
q.insert("b", 0)
print(q.contains("b"), ':',  "the queue now contains b")
print(q.peek(), ':',  "b is now at the front of the queue")
x = q.pop()
print(x, ':',  "we removed b from the queue")
print(q.isEmpty(), ':',  "the queue still nonempty")
print(q.contains("a"), ':',  "the queue still contains a")
print(q.contains("b"), ':',  "the queue does not contain b anymore")
print(q.peek(), ':',  "a is at the front of the queue")
q.insert("c", 5)
print(q.peek(), ':',  "a is still at the front of the queue")
q.update("a", 6)
print(q.peek(), ':',  "after updating a is no longer at the front of the queue")

True : the queue is empty
False : the queue is not empty
True : the queue contains a
False : the queue does not contain a
(1, 'a') : a is the first element in the queue
True : the queue now contains b
(0, 'b') : b is now at the front of the queue
b : we removed b from the queue
False : the queue still nonempty
True : the queue still contains a
False : the queue does not contain b anymore
(1, 'a') : a is at the front of the queue
(1, 'a') : a is still at the front of the queue
(5, 'c') : after updating a is no longer at the front of the queue


### Part 1

In [12]:
from collections import defaultdict
def invert_transition_map(mdp: markov_decision_process.FiniteMarkovDecisionProcess[S, A]) ->\
            Mapping[S, Iterable[S]]:
    '''
    YOUR CODE HERE
    Implement the invert_transition_map method
    '''
    inverted_mapping: Dict[S, Iterable[S]] = {}
    mdp_mapping: Mapping[S, Mapping[A, FiniteDistribution[Tuple[S, float]]]] = mdp.mapping
    
    for currState in mdp_mapping.keys():
        inverted_mapping[currState.state] = set()
    
    for currState in mdp_mapping.keys():
        action_map = mdp_mapping[currState]
        
        possibleNewStates: Set[S] = set()
            
        for dist_map in action_map.values():
            for new_state_and_reward, prob in dist_map:
                new_state = new_state_and_reward[0]
                if prob > 0 and (new_state in mdp_mapping.keys()):
                    inverted_mapping[new_state.state].add(currState.state)       
        
    #raise NotImplementedError
    '''END YOUR CODE'''
    return inverted_mapping




### Part 2

In [42]:
from rl.dynamic_programming import V, S, A, extended_vf

def gaps_value_iteration(
    mdp: markov_decision_process.FiniteMarkovDecisionProcess[S, A],
    gamma: float, 
    gaps: PriorityQueue) -> Iterator[V[S]]:
    '''
    Calculate the value function (V*) of the given MDP by applying the
    update function repeatedly until the values converge.

    '''
    dependency_map = invert_transition_map(mdp)
    v_0: V[S] = {s: 0.0 for s in mdp.non_terminal_states}
        
    def update(v: V[S]) -> V[S]:

        maxState: S = gaps.pop()
            
        maxState_action_map = mdp.mapping[NonTerminal(maxState)]
        
        def bestActionValue(state_action_map): 
            action_value_map: Dict[A, float] = {}
            max_val = float('-inf')
            for dist_map in state_action_map.values():

                reward: float = 0
                future_value: float = 0

                for stateReward, prob in dist_map:
                    reward += prob * stateReward[1]
                    if isinstance(stateReward[0], NonTerminal):
                        future_value += gamma * prob * v[stateReward[0]]

                max_val = max(future_value + reward, max_val)
            
            return max_val
        
        v[NonTerminal(maxState)] = bestActionValue(maxState_action_map)
        
        for dependent_state in dependency_map[maxState]:
            dependent_state_best_val = bestActionValue(mdp.mapping[NonTerminal(dependent_state)])
            stateNewGap = abs(v[NonTerminal(dependent_state)] - dependent_state_best_val)
            if stateNewGap == 0:
                continue
            if not(gaps.isEmpty()) and gaps.contains(dependent_state):
                gaps.update(dependent_state, -stateNewGap)
            else:
                gaps.insert(dependent_state, -stateNewGap)

            
        '''
        YOUR CODE HERE
        perform a single update to v for the state with the largest gap
        update the gaps for any dependent states
        '''
        #raise NotImplementedError 
        '''END YOUR CODE'''
        return v

    
    return iterate(update, v_0)


def gaps_value_iteration_result(
    mdp: FiniteMarkovDecisionProcess[S, A],
    gamma: float
) -> Tuple[V[S], FiniteDeterministicPolicy[S, A]]:
    
    gaps = PriorityQueue()

    v: V[S] = {s: 0.0 for s in mdp.non_terminal_states}
    
    def bestActionValue(state_action_map): 
        action_value_map: Dict[A, float] = {}

        for action in state_action_map.keys():
            dist_map = state_action_map[action]

            reward: float = 0
            future_value: float = 0

            for stateReward, prob in dist_map:
                reward += prob * stateReward[1]
                if isinstance(stateReward[0], NonTerminal):
                    future_value += gamma * prob * v[stateReward[0]]

            action_value_map[action] = future_value + reward

        return max(action_value_map.values())
    
    for s in mdp.non_terminal_states:
        sBestVal = -abs(bestActionValue(mdp.mapping[s]))
        if not(gaps.isEmpty()) and sBestVal < gaps.peek()[0]:
            gaps.pop()
            gaps.insert(s.state, sBestVal)
        if not(gaps.isEmpty()) and sBestVal == gaps.peek()[0] and not(gaps.contains(s.state)):
            gaps.insert(s.state, sBestVal)
        elif gaps.isEmpty():
            gaps.insert(s.state, sBestVal)
            
    '''
    YOUR CODE HERE
    instantiate the value function and populate the gaps
    ''' 
    #raise NotImplementedError
    ''' END YOUR CODE ''' 
    
    def criterion(x,y):
        THRESHOLD = 1e-5
        return (gaps.isEmpty()) or (abs(gaps.peek()[0]) < THRESHOLD)
        ''' END YOUR CODE ''' 
        
    opt_vf: V[S] = converged(
        gaps_value_iteration(mdp, gamma, gaps),
        done= criterion 
    )
        
    opt_policy: markov_decision_process.FiniteDeterministicPolicy[S, A] = dynamic_programming.greedy_policy_from_vf(
        mdp,
        opt_vf,
        gamma
    )

    return opt_vf, opt_policy


# Do not change the code below here, just run it

## Calculating the VF for a maze with sparse rewards

In [43]:
underlying_maze = grid_maze.Maze(50, 50)
maze_mdp = grid_maze.GridMazeMDP_Sparse(underlying_maze, 0, 0)
#print(maze_mdp)

#### printing the runtime for the calculation

In [44]:
start = time()
v1_res = gaps_value_iteration_result(maze_mdp, 0.9)
print(time() - start)

0.14131689071655273


In [45]:
start = time()
v2_res = dynamic_programming.value_iteration_result(maze_mdp, 0.9)
#print(v2_res)
print(time() - start)

1.8211369514465332


#### confirming that the value functions are identical

In [46]:
assert v1_res[1] == v2_res[1]

## Calculating the VF for a maze with dense rewards

In [47]:
maze_mdp = grid_maze.GridMazeMDP_Dense(underlying_maze, 0, 0)

#### printing the runtime for the calculation

In [48]:
start = time()
v1_res = gaps_value_iteration_result(maze_mdp, 1)
print(time() - start)

3.4453179836273193


In [49]:
start = time()
v2_res = dynamic_programming.value_iteration_result(maze_mdp, 1)
print(time() - start)

1.8264849185943604


#### confirming that the value functions are identical

In [50]:
assert v1_res[1] == v2_res[1]